## import

In [1]:
import numpy as np
import os, random
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 
from sklearn.model_selection import KFold 
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, Dropout
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import Lambda
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

C:\Users\HW-steve\miniconda3\envs\points\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HW-steve\miniconda3\envs\points\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HW-steve\miniconda3\envs\points\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HW-steve\miniconda3\envs\points\lib\site-packages

## random seed

In [2]:
ramdom_seed = 5198
tf.random.set_random_seed(ramdom_seed)
np.random.seed(ramdom_seed)
random.seed(ramdom_seed)
os.environ['PYTHONHASHSEED'] = str(ramdom_seed)

## GPU growth

In [3]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

## Preprocessing function

In [4]:
def mat_mul(A, B):
    return tf.matmul(A, B)

def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)

def rotate_point_cloud(batch_data):
    rotated_data = np.zeros(batch_data.shape, dtype=np.float32)
    for k in range(batch_data.shape[0]):
        rotation_angle = np.random.uniform() * 2 * np.pi
        cosval = np.cos(rotation_angle)
        sinval = np.sin(rotation_angle)
        rotation_matrix = np.array([[cosval, 0, sinval],
                                    [0, 1, 0],
                                    [-sinval, 0, cosval]])
        shape_pc = batch_data[k, ...]
        rotated_data[k, ...] = np.dot(shape_pc.reshape((-1, 3)), rotation_matrix)
    return rotated_data

def jitter_point_cloud(batch_data, sigma=0.001, clip=0.005):
    B, N, C = batch_data.shape
    assert(clip > 0)
    jittered_data = np.clip(sigma * np.random.randn(B, N, C), -1 * clip, clip)
    jittered_data += batch_data
    return jittered_data

In [43]:
def model_create():
    # number of points in each sample
    num_points = 10000

    # number of categories
    k = 3

    # define optimizer
    adam = optimizers.Adam(lr=0.001, decay=0.7)

    # ------------------------------------ Pointnet Architecture
    # input_Transformation_net
    input_points = Input(shape=(num_points, 3))
    x = Convolution1D(64, 1, activation='relu', input_shape=(num_points, 3))(input_points)
    x = Convolution1D(128, 1, activation='relu')(x)
    x = Convolution1D(1024, 1, activation='relu')(x)
    x = MaxPooling1D(pool_size=num_points)(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(9, weights=[np.zeros([256, 9]), np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32)])(x)
    input_T = Reshape((3, 3))(x)

    # forward net
    g = Lambda(mat_mul, arguments={'B': input_T})(input_points)
    g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
    g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)

    # feature transform net
    f = Convolution1D(64, 1, activation='relu')(g)
    f = Convolution1D(128, 1, activation='relu')(f)
    f = Convolution1D(1024, 1, activation='relu')(f)
    f = MaxPooling1D(pool_size=num_points)(f)
    f = Dense(512, activation='relu')(f)
    f = Dense(256, activation='relu')(f)
    f = Dense(64 * 64, weights=[np.zeros([256, 64 * 64]), np.eye(64).flatten().astype(np.float32)])(f)
    feature_T = Reshape((64, 64))(f)

    # forward net
    g = Lambda(mat_mul, arguments={'B': feature_T})(g)
    g = Convolution1D(64, 1, activation='relu')(g)
    g = Convolution1D(128, 1, activation='relu')(g)
    g = Convolution1D(1024, 1, activation='relu')(g)

    # global_feature
    global_feature = MaxPooling1D(pool_size=num_points)(g)

    # point_net_cls
    c = Dense(512, activation='relu')(global_feature)
    c = Dropout(rate=0.2)(c)
    c = Dense(256, activation='relu')(c)
    c = Dropout(rate=0.2)(c)
    c = Dense(k, activation='sigmoid')(c)
    prediction = Flatten()(c)
    # --------------------------------------------------end of pointnet

    # print the model summary
    model = Model(inputs=input_points, outputs=prediction)
    model.compile(optimizer='Adam', loss='MSE', metrics=['accuracy'])
#     print(model.summary())
    
    return model

In [44]:
data_path = 'D:/DATA/data_2020-05-21_knu_deid/data_ply_np.npy'

In [45]:
train_np = np.load(data_path)
# train_jitter = jitter_point_cloud(train_np, sigma = 0.1, clip = 1)

label_path = 'D:/DATA/data_2020-05-21_knu_deid/angle_class_labels.csv'
df = pd.read_csv(label_path, delimiter=',')
angle_label = df['Angle Class Model'].tolist() 
angle_label_np = np.array(angle_label)
angle_label_np = angle_label_np -1

In [46]:
print('data shape :', train_np.shape, 'labels shape :', angle_label_np.shape)

data shape : (129, 10000, 3) labels shape : (129,)


In [47]:
# def data_generator(train_np, anger_label_np, batch_size, aug = True):
#     idx = 0
#     idx_list = list(range(train_np.shape[0])) # random shuffle index generation
#     anger_label_cate = to_categorical(anger_label_np -1)
    
#     while 1:        
#         if idx > len(idx_list) - batch_size:
#             tmp_list = idx_list[idx:]
#             idx = 0
#         else:
#             tmp_list = idx_list[idx:idx+batch_size] # end of list batch 
#             idx = idx + batch_size
            
#         batch_points = train_np[tmp_list, ...]
#         batch_labels = anger_label_cate[tmp_list, ...]

#         if aug == True:
#             train_points_jitter = jitter_point_cloud(batch_points, sigma = 0.1, clip = 1)
#         else:
#             train_points_jitter = batch_points
                
#         yield train_points_jitter, batch_labels

In [48]:
kfold = StratifiedKFold(n_splits=10, random_state=ramdom_seed) 

C:\Users\HW-steve\miniconda3\envs\points\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
history_all = []
eval_all = []
for train_index, test_index in kfold.split(train_np, angle_label_np): 
    model = model_create()
    train_input = train_np[train_index]
    train_label = to_categorical(angle_label_np[train_index])
    test_input = train_np[test_index]
    test_label = to_categorical(angle_label_np[test_index])
    history = model.fit(train_input, train_label, batch_size = 64, epochs = 200)
    history_all.append(history)
    eval_ = model.evaluate(test_input, test_label, batch_size = 64)
    eval_all.append(eval_)

Epoch 1/200
116/116 [==============================] - 2s 14ms/sample - loss: 0.2681 - acc: 0.3362
Epoch 2/200
116/116 [==============================] - 1s 9ms/sample - loss: 0.2731 - acc: 0.4397
Epoch 3/200
116/116 [==============================] - 1s 9ms/sample - loss: 0.2816 - acc: 0.3017
Epoch 4/200
116/116 [==============================] - 1s 9ms/sample - loss: 0.2666 - acc: 0.3534
Epoch 5/200
116/116 [==============================] - 1s 9ms/sample - loss: 0.2710 - acc: 0.3621
Epoch 6/200
116/116 [==============================] - 1s 9ms/sample - loss: 0.2637 - acc: 0.3448
Epoch 7/200
116/116 [==============================] - 1s 9ms/sample - loss: 0.2460 - acc: 0.4569
Epoch 8/200
116/116 [==============================] - 1s 9ms/sample - loss: 0.2354 - acc: 0.3793
Epoch 9/200
116/116 [==============================] - 1s 9ms/sample - loss: 0.2234 - acc: 0.4569
Epoch 10/200
116/116 [==============================] - 1s 9ms/sample - loss: 0.2229 - acc: 0.3879
Epoch 11/200
116/1

In [ ]:
eval_all

In [ ]:
# y_loss = history_all.history['loss']
# y_acc = history_all.history['acc']

x_len = np.arange(len(history_all[0].history['loss']))

plt.figure(figsize = (20, 10))
for i in range(len(history_all)):
    plt.plot(x_len, history_all[i].history['loss'], marker='.', label= f"fold {i}")
    # plt.plot(x_len, history_all[i].history['acc'], marker='.', c='red', label="Train-set Accuracy")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylim((0, 0.4))
plt.show()

In [ ]:
# y_loss = history_all.history['loss']
# y_acc = history_all.history['acc']

x_len = np.arange(len(history_all[0].history['loss']))

plt.figure(figsize = (20, 10))
for i in range(len(history_all)):
#     plt.plot(x_len, history_all[i].history['loss'], marker='.', c='blue', label= f"fold {i}")
    plt.plot(x_len, history_all[i].history['acc'], marker='.', label=f"fold {i}")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylim((0, 1))
plt.show()